In [1]:
from bs4 import BeautifulSoup as soup  # HTML data structure
from urllib.request import urlopen, Request  # Web client

In [10]:
def fetchAll(page_soup, housePageUrl):
    data = ""
    AboutHouseInfo = fetchAboutHouseInfo(page_soup)
    HousePrimaryInfo = fetchHousePrimaryInfo(page_soup)
    EstimatedPriceInfo = fetchEstimatedPriceInfo(page_soup)
    HouseBasicInfo = fetchHouseBasicInfo(page_soup)
    FeatureInfo = fetchFeatureInfo(page_soup)
    data = "{},{},{},{},{},{}".format(housePageUrl, AboutHouseInfo, HousePrimaryInfo, EstimatedPriceInfo, HouseBasicInfo, FeatureInfo)
    print(len(data.split(',')))
    print(data)
    f = open("redfin.csv", "a")
    f.write(data+"\n")
    f.close()
    
    
def fetchAboutHouseInfo(page_soup):
    data="@#$"
    try:
        abouthouse = page_soup.find("div", {"class": "house-info-container"}).find("div", {"class": "remarks"}).div.p
        data+=str(abouthouse.text).replace(",",".")
        data = data.replace("@#$","")
    except:
        print("Not Found")
        abouthouse = page_soup.find("div", {"class": "house-info-container"}).div.div.div.findAll('span')
        print(len(abouthouse))
        for info in abouthouse:
            try:
                data += "." + str(info.text).replace(",",".")
            except:
                continue
        data = data.replace("@#$.","")
    return data
    
def fetchHousePrimaryInfo(page_soup):
    data="@#$"
    Dict = {'HOA Dues': '-', 'Property Type': '-', 'Style': '-', 'Floor Number': '-', 'Stories': '-', 'View(s)': '-', 'Community': '-', 'County': '-', 'MLS#': '-'}
    colList = ['HOA Dues', 'Property Type', 'Style', 'Floor Number', 'Stories', 'View(s)', 'Community', 'County', 'MLS#']    
    try:
        housprimaryInfo= page_soup.find("div", {"class": "keyDetailsList"}).findAll("div", {"class": "keyDetail font-weight-roman font-size-base"})
        for i in range(0,len(housprimaryInfo)):
            info = housprimaryInfo[i].findAll("span");
            try:
                if(Dict[str(info[0].text)]):
                    Dict[str(info[0].text)] = str(info[1].text).replace(",","")
            except:
                continue
        for j in colList:
            data += ","+str(Dict[str(j)])
        data = data.replace("@#$,","")
        return data
    except:
        for i in colList:
            data += ","+str(Dict[str(i)])
        data = data.replace("@#$,","")
        return data
        
def fetchHouseBasicInfo(page_soup):
    data="@#$"
    Dict = {'Beds': '-', 'Baths': '-', 'Sq. Ft.': '-', 'Year Built': '-', 'Year Renovated': '-', 'Garage': '-'}
    colList = ['Beds', 'Baths', 'Sq. Ft.', 'Year Built', 'Year Renovated', 'Garage']
    try:
        basicInfo = page_soup.find("div", {"class":"basic-info"})
        basicInfoTable = basicInfo.find("div", {"class":"median-values"})
        basicInfoTableRows = basicInfoTable.find("div", {"class":"facts-table"}).findAll("div", {"class":"table-row"})
        for i in range(0,len(basicInfoTableRows)):
            try:
                infoLabel = str(basicInfoTableRows[i].find('span', {"class":"table-label"}).text)
                infoValue = str(basicInfoTableRows[i].find('div', {"class":"table-value"}).text)
                if(Dict[infoLabel]):
                    Dict[infoLabel] = infoValue.replace(",","")
            except:
                continue
        for j in colList:
            data += ","+str(Dict[str(j)])
        data = data.replace("@#$,","")
        return data
    except:
        for j in colList:
            data += ","+str(Dict[str(j)])
        data = data.replace("@#$,","")
        return data

def fetchEstimatedPriceInfo(page_soup):
    data=""
    try:
        estimatedPrice = str(page_soup.find("div", {"class": "RedfinEstimateValueHeader"}).div.text)
        data = estimatedPrice.replace(",", "")
        return data
    except:
        return data




def fetchFeatureInfo(page_soup):
    Dict = {'room_info': '-', 'fireplace_Info': '-', 'heating_info': '-', 'parking_info': '-', 'property_info': '-', 'exterior_info': '-'}
    colList = ['room_info', 'fireplace_Info', 'heating_info', 'parking_info', 'property_info', 'exterior_info']

    data="@#$"
    try:
        FeaturesInfo = page_soup.find("div", {"class":"amenities-container"})
        #print(FeaturesInfo)
        FeatureTitle = FeaturesInfo.findAll("div", {"class":"super-group-title"})
        FeatureContent = FeaturesInfo.findAll("div", {"class":"super-group-content"})

        #print(FeatureTitle)
        #print(FeatureContent)
        for i in range(0,len(FeatureTitle)):
            #print(title.text)
            if "interior" in str(FeatureTitle[i].text).lower():
                amenityGroups = FeatureContent[i].findAll("div", {"class":"amenity-group"})
                for j in range(0,len(amenityGroups)):
                    if "fireplace" in str(amenityGroups[j].text).lower():
                        val = str(Dict["fireplace_Info"].replace("-",""))
                        val = val + amenityGroups[j].text
                        Dict["fireplace_Info"] = val.replace(",","")
                        #print(amenityGroups[j].text)
                    if ("room" in str(amenityGroups[j].text).lower()):
                        #print(amenityGroups[j].text)
                        val = str(Dict["room_info"].replace("-",""))
                        val = val + amenityGroups[j].text
                        Dict["room_info"] = val.replace(",","")
                    if "heating" in str(amenityGroups[j].text).lower():
                        #print(amenityGroups[j].text)
                        val = str(Dict["heating_info"].replace("-",""))
                        val = val + amenityGroups[j].text
                        Dict["heating_info"] = val.replace(",","")

            if ("parking" in str(FeatureTitle[i].text).lower() or "property" in str(FeatureTitle[i].text).lower() or "garage" in str(FeatureTitle[i].text).lower() or "exterior" in str(FeatureTitle[i].text).lower()):
                amenityGroups = FeatureContent[i].findAll("div", {"class":"amenity-group"})
                for j in range(0,len(amenityGroups)):
                    if "parking" in str(amenityGroups[j].text).lower():
                        #print(amenityGroups[j].text)
                        val = str(Dict["parking_info"].replace("-",""))
                        val = val + amenityGroups[j].text
                        Dict["parking_info"] = val.replace(",","")
                    if "property" in str(amenityGroups[j].text).lower():
                        #print(amenityGroups[j].text)
                        val = str(Dict["property_info"].replace("-",""))
                        val = val + amenityGroups[j].text
                        Dict["property_info"] = val.replace(",","")
                    if "exterior" in str(amenityGroups[j].text).lower():
                        #print(amenityGroups[j].text)
                        val = str(Dict["exterior_info"].replace("-",""))
                        val = val + amenityGroups[j].text
                        Dict["exterior_info"] = val.replace(",","")

        print(Dict)
        for j in colList:
            data += ","+str(Dict[str(j)]).replace(",","")
        data = data.replace("@#$,","")
        return data
    except:
        print(Dict)
        for j in colList:
            data += ","+str(Dict[str(j)]).replace(",","")
        data = data.replace("@#$,","")
        return data

In [ ]:
# URl to web scrap from.
# in this example we web scrap graphics cards from Newegg.com

print("------------------------------------------------------------------------------------")
#set houseid value where you want to start
houseId = 1;
#set the records value , how much record you want to scrap
records=10000;
url = "https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/"
headers="HouseId,AboutHouse,HOA_Dues,PropertyType,Style,Floors,Stories,View,Community,County,MLS_No,RealEstateEstimate,Beds,Baths,Sq_Ft,YearBuilt,YearRenovated,Garage"

for i in range(records):
    page_url = url + str(houseId)
    print(page_url)
    req = Request(page_url, headers={'User-Agent': 'Mozilla/5.0'})
    # opens the connection and downloads html page from url
    uClient = urlopen(req)
    page_soup = soup(uClient.read(), "html.parser")
    uClient.close()
    fetchAll(page_soup, page_url)
    print("------------------------------------------------------------------------------------")
    houseId = houseId + 1;

------------------------------------------------------------------------------------
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6669
Not Found
5
{'room_info': 'Bathroom Information# of Full Baths: 2# of 1/2 Baths: 1', 'fireplace_Info': 'Fireplace InformationHas Fireplace', 'heating_info': '-', 'parking_info': 'Parking & Garage Information# of Parking Spaces: 2', 'property_info': 'Property InformationSubdivision Name: BARCLAY SQUARELiving Sq. Ft: 1472# of Units: 64# of Stories: 2Legal Description: BARCLAY SQUARE CONDOMINIUM PCT OF VALUE 1.5775 PLAT BLOCK: PLAT LOT:', 'exterior_info': 'Exterior InformationPhyiscal Condition: FairConstruction Type: WoodBuilding Construction Quality: GoodBuilding Type: DuplexBuilding Improvement Type: Townhouse'}
24
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6669,3639 255th Ln SE #36 is a condo in Issaquah. WA 98029. This 1.472 square foot condo features 2 bedrooms and 2.5 bathrooms. This property w

Not Found
6
{'room_info': 'Bathroom Information# of Full Baths: 1', 'fireplace_Info': 'Fireplace InformationHas Fireplace', 'heating_info': '-', 'parking_info': 'Parking & Garage InformationParking Type: Shared/Common', 'property_info': 'Property InformationSubdivision Name: SUNSET GARDEN CONDOLiving Sq. Ft: 795# of Units: 60# of Stories: 3Legal Description: SUNSET GARDEN CONDOMINIUM PCT OF VALUE 1.53 P-36 PLAT BLOCK: PLAT LOT:', 'exterior_info': 'Exterior InformationPhyiscal Condition: FairConstruction Type: WoodBuilding Construction Quality: AverageBuilding Type: Single FamilyBuilding Improvement Type: Flat'}
24
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6673,949 Aberdeen Ave NE Unit B305 is a condo in Renton. WA 98056. This 795 square foot condo features 2 bedrooms and 1 bathroom. This property was built in 1991 and last sold for $77.950. .Based on Redfin's Renton data. we estimate the home's value is $232.268. Comparable nearby homes include 949 Aberdee

Not Found
6
{'room_info': 'Bathroom Information# of Full Baths: 1', 'fireplace_Info': 'Fireplace InformationHas Fireplace', 'heating_info': '-', 'parking_info': 'Parking & Garage Information# of Parking Spaces: 1', 'property_info': 'Property InformationSubdivision Name: SUNWOOD PH I CONDOLiving Sq. Ft: 754# of Units: 178# of Stories: 2Legal Description: SUNWOOD CONDOMINIUM PCT OF VALUE .411 PLAT BLOCK: PLAT LOT:', 'exterior_info': 'Exterior InformationPhyiscal Condition: FairConstruction Type: WoodBuilding Construction Quality: AverageBuilding Type: Single FamilyBuilding Improvement Type: Flat'}
24
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6677,15255 Sunwood Blvd Unit A41 is a condo in Tukwila. WA 98188. This 754 square foot condo features 1 bedroom and 1 bathroom. This property was built in 1980 and last sold for $47.000. .Based on Redfin's Tukwila data. we estimate the home's value is $171.467. Comparable nearby homes include 15209 Sunwood Blvd Unit B44.

{'room_info': 'Bedroom Information# of Bedrooms (Main): 2Master Bedroom on Main LevelBathroom Information# of Baths (Full): 1# of Baths (Total): 1.75# of Baths (3/4): 1# of Main Baths (Full): 1# of Main Baths (3/4): 1Room InformationKitchen with Eating Space on Main LevelDining Room on Main LevelLiving Room on Main LevelFamily Room on Main LevelUtility Room on Main Level', 'fireplace_Info': 'Fireplace InformationWood# of Fireplaces: 1# of Fireplaces (Main): 1', 'heating_info': 'Heating & CoolingBaseboard HeatWater Heater: Electric', 'parking_info': 'Parking Information# of Assigned Spaces: 1Carport', 'property_info': 'Property FeaturesAlarm System Balcony/Deck/Patio Ground Floor Insulated Windows Master Bath YardProperty Information# of Access Stairs: 12Entry on Main LevelPreliminary Title Ordered: YesTax ID #: 8141400250Plat/Subdivision: Sunwood Condo', 'exterior_info': 'Building InformationStucco Exterior Wood Exterior Wood Products ExteriorComposition RoofStorage Location: PatioEffe

{'room_info': 'Bedroom Information# of Bedrooms (Upper): 2Master Bedroom on Upper LevelBathroom Information# of Baths (Full): 1# of Baths (Total): 1.5# of Baths (1/2): 1# of Upper Baths (Full): 1# of Main Baths (1/2): 1Room InformationKitchen without Eating Space on Main LevelDining Room on Main LevelLiving Room on Main LevelUtility Room on Upper Level', 'fireplace_Info': '-', 'heating_info': 'Heating & CoolingBaseboard Heat Free-Standing Stove HeatWater Heater: Electric/Cupboard', 'parking_info': 'Parking InformationCarportCommunity Information# of Units in Complex: 124Athletic Court Club House Exercise Room Game/Recreation Room Outside Entrance Outdoor Pool RV Parking Sauna Security Gate TrailsCats Only', 'property_info': 'Property FeaturesBalcony/Deck/Patio Skylights Vaulted CeilingProperty InformationEntry on Main LevelPreliminary Title Ordered: YesTax ID #: 7804170490Plat/Subdivision: Sixty-01', 'exterior_info': 'Building Information# of Units: 16Remodeled/UpdatedWood ExteriorComp

Not Found
6
{'room_info': 'Bathroom Information# of Full Baths: 1', 'fireplace_Info': 'Fireplace InformationHas Fireplace', 'heating_info': '-', 'parking_info': 'Parking & Garage Information# of Parking Spaces: 1', 'property_info': 'Property InformationSubdivision Name: LAKE UNION HOUSE CONDOLiving Sq. Ft: 612# of Units: 27# of Stories: 4Legal Description: LAKE UNION HOUSE A CONDOMINIUM PCT OF VALUE 3.11 PLAT BLOCK: PLAT LOT:', 'exterior_info': 'Exterior InformationPhyiscal Condition: FairConstruction Type: WoodBuilding Construction Quality: GoodBuilding Type: Single FamilyBuilding Improvement Type: Flat'}
24
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6689,949 N 35th St #305 is a condo in Seattle. WA 98103. This 612 square foot condo features 1 bedroom and 1 bathroom. This property was built in 1978 and last sold for $98.950. .Based on Redfin's Seattle data. we estimate the home's value is $353.265. Comparable nearby homes include 949 N 35th St #100. 949 N 

Not Found
6
{'room_info': 'Bathroom Information# of Full Baths: 2', 'fireplace_Info': 'Fireplace InformationHas Fireplace', 'heating_info': '-', 'parking_info': 'Parking & Garage Information# of Parking Spaces: 1', 'property_info': 'Property InformationSubdivision Name: MORGAN COURT CONDOLiving Sq. Ft: 1553# of Units: 43# of Stories: 2Legal Description: MORGAN COURT CONDOMINIUM PCT UND INT 2.56 PLAT BLOCK: PLAT LOT:', 'exterior_info': 'Exterior InformationPhyiscal Condition: FairConstruction Type: WoodBuilding Construction Quality: AverageBuilding Type: Duplex'}
24
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6693,18700 109 Ln SE #18704 is a condo in Renton. WA 98055. This 1.553 square foot condo features 3 bedrooms and 2 bathrooms. This property was built in 2000 and last sold for $305.000. .Based on Redfin's Renton data. we estimate the home's value is $424.738. Comparable nearby homes include 5012 Lake Ave S Unit C. 4803 Shattuck Pl S Unit DD101. and 4820 

Not Found
5
{'room_info': 'Bathroom Information# of Full Baths: 1', 'fireplace_Info': 'Fireplace InformationHas Fireplace', 'heating_info': '-', 'parking_info': 'Parking & Garage Information# of Parking Spaces: 1Parking Type: Basement GarageGarage / Carport Type: Basement', 'property_info': 'Property InformationSubdivision Name: ELEKTRALiving Sq. Ft: 724# of Units: 200# of Stories: 16Legal Description: ELEKTRA CONDOMINIUM PCT UND INT 0.6260 PLAT BLOCK: PLAT LOT:', 'exterior_info': 'Exterior InformationPhyiscal Condition: FairConstruction Type: ConcreteBuilding Construction Quality: AverageBuilding Type: Single FamilyBuilding Improvement Type: Flat'}
24
https://www.redfin.com/WA/Seattle/2030-Western-Ave-98121/unit-P255/home/6697,1400 Hubbell Pl #1109 is a condo in Seattle. WA 98101. This 724 square foot condo features 2 bedrooms and 1 bathroom. This property was built in 1957 and last sold for $177.950. .Based on Redfin's Seattle data. we estimate the home's value is $510.300. Comparabl